## Общая характеристика API
Аббревиатура расшифровывается как Application Programming Interface, или интерфейс для программирования приложений.
В случае веб-приложений, API может отдавать данные в отличном от стандартного HTML формате, благодаря чему им удобно пользоваться при написании собственных приложений. Сторонние общедоступные API чаще всего отдают данные в одном из двух форматов: XML или JSON.

На основе API строятся такие вещи, как карты 2GIS, всевозможные мобильные и десктопные клиенты для Twitter и Vkontakte. Все их функции стали возможными именно благодаря тому, что соответствующие сервисы имеют качественные и детально документированные API.

API VK описан в документации https://vk.com/dev и более конкретно https://vk.com/dev/api_requests.

Можно попробовать сделать запрос из браузера вручную https://api.vk.com/method/getProfiles?uid=66748. В качестве ответа на этот запрос мы получим ответ в формате json:

`{"response":[{"uid":66748,"first_name":"Oleg","last_name":"Illarionov","hidden":1}]}`



> P.S.:

> Почитайте про формат строки [URL](https://ru.wikipedia.org/wiki/URL).

>> Параметры – строка запроса с передаваемыми на сервер (методом GET) параметрами. Начинается с символа ?, разделитель параметров — знак &. Пример: ?параметр_1=значение_1&параметр_2=значение_2&параметр3=значение_3

> Почитайте про формат [JSON](https://ru.wikipedia.org/wiki/JSON).

## Создание приложения

Для того что бы использовать все возможности VK API нужно получить access token аккаунта. Для этого Вам потребуется [создать Standalone-приложение](https://vk.com/editapp?act=create).

После того как мы создали приложение Вы можете найти его разделе [Приложения](https://vk.com/apps?act=manage).

## Приватный токен

Многие методы VK API предполагают наличие приватного токена, который необходимо передать в качетсве параметра при выполнении запроса. Процесс получения токена описан в документации: https://vk.com/dev/access_token

> Осторожно! Токен называется приватным не просто так. Обладающий им человек может производить самые разные действия от Вашего имени. Не говорите его никому.

Если вкратце, то Вам поднадобится id Вашего приложения и [список прав доступа](https://vk.com/dev/permissions), которые Вы хотите предоставить пользователю API. Затем необходимо указать эти данные в качестве параметров в URL следующего формата
https://oauth.vk.com/authorize?client_id={APP_ID}&scope={APP_PERMISSIONS}&response_type=token, подтвердить своё намерение предоставить доступ и скопировать токент из поля URL открывшегося окна.

Покажем на примере:

In [1]:
APP_ID = 6217902
PERMISSIONS = ""
AUTH_URL = f"https://oauth.vk.com/authorize?client_id={APP_ID}&scope={PERMISSIONS}&response_type=token"
AUTH_URL

'https://oauth.vk.com/authorize?client_id=6217902&scope=&response_type=token'

Пройдя по этой ссылки мы попадём настраницу с адресом https://oauth.vk.com/blank.html#access_token=5614afdcc2bcd42cea3d9c5edc130101dd4be5339b484131870dc12337e5b74b94411de69f0996379dd6b&expires_in=86400&user_id=18779389, где `5614afdcc2bcd42cea3d9c5edc130101dd4be5339b484131870dc12337e5b74b94411de69f0996379dd6b` — искомый access token. Сохраним его.

In [2]:
TOKEN = "7b061b939e04be316155ed2539c990417c37cf3da5eb3cf4112bdcf847a7861f10c6b0d03af37adb18942"

## Выполнение запросов к VK API

После получения приватного токена можно смело выполнять запросы к API, используя нужные Вам [методы](https://vk.com/dev/methods). Формат запроса следующий: https://api.vk.com/method/METHOD_NAME?PARAMETERS&access_token=ACCESS_TOKEN. Например, для получения информации о пользователе c id 777 нужно выполнить следующй запрос:

In [5]:
import requests

res = requests.get(
    "https://api.vk.com/method/users.get",
    params={"user_ids": 777,
            "fields": "nickname, screen_name, sex, bdate, city, country, timezone, counters, photo_medium",
            "access_token": TOKEN,
            "v":5.52}).json()
res

{'response': [{'id': 777,
   'first_name': 'Teodor',
   'last_name': 'Master',
   'sex': 2,
   'nickname': '†',
   'screen_name': 'id777',
   'bdate': '23.10',
   'city': {'id': 22168, 'title': 'Amsterdam'},
   'country': {'id': 139, 'title': 'Netherlands'},
   'photo_medium': 'https://pp.userapi.com/c606428/v606428777/58d0/8seTeA35knk.jpg',
   'counters': {'albums': 8,
    'videos': 34,
    'audios': 360,
    'notes': 8,
    'photos': 146,
    'groups': 7,
    'gifts': 130,
    'friends': 5136,
    'online_friends': 73,
    'mutual_friends': 0,
    'user_photos': 3,
    'followers': 5294,
    'subscriptions': 132,
    'pages': 0}}]}

## Ограничения VK API
**Ограничение количества запросов через vk api — не более трёх запросов в секунду.**

>There can be maximum 3 requests to API methods per second from a client. 

>Maximum amount of server requests depends on the app's users amount. 
If an app has less than 10 000 users, 5 requests per second, up to 100 000 – 8 requests, up to 1 000 000 – 20 requests, 1 000 000+ – 35 requests. 

>If one of this limits is exceeded, the server will return following error: 'Too many requests per second'. 

>If your app's logic implies many requests in a row, check the execute method. 

>Except the frequency limits there are quantitative limits on calling the methods of the same type. By obvious reasons we don't provide the exact limits info. 

>On excess of a quantitative limit access to a particular method will require captcha (see captcha_error). After that it may be temporarily limited (in this case the server doesn't answer on particular method's requests but easily processes any other requests).

Сделать паузу при выполнении какой-либо операции в Python можно с помощью функции `slepp` из модуля `time`, которой необходимо передать количество секунд, на которые присотановится выполнение программы:

In [106]:
import time

for i in range(5):
    time.sleep(.5)
    print(i)

0
1
2
3
4


## Ошибки при работе с VK API

Достаточно часто VK API возвращает [ошибки](https://vk.com/dev/errors), которые необходимо отлавливать. В ином случае к концу сбора данных Вы можете обнаружить, что с таким трудом собранная база данных абсолютно непригода к работе, поскольку вместо информации о респонденте Вы имеет следующие данные:

Также существует множество всяких другх тонкостей при работе с VK API. Например, для получения списка друзей пользователя необходимо использовать метод [`friends.get`](https://vk.com/dev/friends.get), который может возвращать как просто список друзей, так и детальную информацию о каждом друге, в зависимости от того, указан ли параметр `fields` (если не указан, возвращает просто список id). Причем если параметр `fields` указан, то за один запрос Вы не можете получить информацию о более, чем 5000 человек.

## Самостоятельная работа

Постройте эго-сеть друзей пользователя с id 777 в формате Adjacency lists. Найдите в этой сети пользователя с найбольшей степенью центральности (Degree centrality), исключая, конечно, самого пользователя с id 777.

> 
![](https://courses.cs.washington.edu/courses/csep521/99sp/lectures/lecture01/img029.JPG)

In [9]:
friends = requests.get(
    "https://api.vk.com/method/friends.get",
    params={"user_id": 777, "count": 10000,
            "access_token": TOKEN, "v":5.52}).json()

In [10]:
print(friends)

{'response': {'count': 5136, 'items': [500, 1897, 2328, 3027, 3828, 3972, 6111, 6622, 9739, 14729, 15140, 15266, 17320, 17990, 18271, 18745, 19506, 22622, 22987, 24003, 24548, 27089, 28070, 28773, 34006, 34642, 38267, 38968, 42555, 43357, 44730, 46148, 46353, 46705, 46984, 47195, 49508, 51535, 52116, 54957, 57915, 58081, 59086, 61450, 62789, 66566, 68452, 73492, 73574, 75933, 76682, 81535, 81905, 82017, 82991, 85320, 85504, 85727, 85801, 87600, 88076, 88088, 89906, 90155, 90828, 92059, 92454, 92527, 92642, 93218, 93276, 94732, 97222, 98046, 98576, 99759, 101025, 101066, 101335, 101979, 102572, 103085, 103191, 103623, 103799, 105285, 108976, 109293, 112262, 113083, 115204, 115804, 115929, 116909, 117242, 117343, 117860, 119355, 122372, 123034, 123850, 124357, 127824, 128216, 132035, 132187, 132935, 135632, 136722, 137116, 137658, 139964, 141760, 141934, 142313, 142436, 143127, 143207, 143863, 144039, 145063, 152964, 153012, 155280, 156999, 157708, 158148, 158765, 159662, 160550, 160974,

In [14]:
f_list = []
for friend in friends["response"]["items"]:
    res = requests.get(
    "https://api.vk.com/method/users.get",
    params={
        "user_ids": friend,
        "fields": "nickname, screen_name, sex, bdate, city, country, timezone, photo, photo_medium",
        "access_token": TOKEN, "v":5.52
    }).json()
    f_list.append(res)
    

In [15]:
f_list

[{'response': [{'id': 500,
    'first_name': 'DELETED',
    'last_name': '',
    'deactivated': 'deleted',
    'sex': 0,
    'photo': 'https://vk.com/images/deactivated_50.png',
    'photo_medium': 'https://vk.com/images/deactivated_100.png'}]},
 {'response': [{'id': 1897,
    'first_name': 'Sveta',
    'last_name': 'Makerova',
    'sex': 1,
    'nickname': '',
    'screen_name': 'id1897',
    'bdate': '3.6',
    'city': {'id': 2, 'title': 'Saint Petersburg'},
    'country': {'id': 1, 'title': 'Russia'},
    'photo': 'https://pp.userapi.com/c840131/v840131386/13492/X55r2ubIt64.jpg',
    'photo_medium': 'https://pp.userapi.com/c840131/v840131386/13491/xVxA_tw_GQw.jpg'}]},
 {'response': [{'id': 2328,
    'first_name': 'Dmitry',
    'last_name': 'Shpion',
    'deactivated': 'banned',
    'sex': 2,
    'photo': 'https://vk.com/images/deactivated_50.png',
    'photo_medium': 'https://vk.com/images/deactivated_100.png'}]},
 {'response': [{'id': 3027,
    'first_name': 'Ksyusha',
    'last_na

In [95]:
import time
for i in range(10):
    print(i)
    time.sleep(1)

0
1
2
3
4
5
6
7
8
9


In [69]:
def split_list(big_list, n=100):
    split_times = len(big_list) // n + 1
    for i in range(split_times):
        start = i*n
        end = (i+1)*n
        yield big_list[start:end]
    
def get_users(ids):
    if isinstance(ids, list):
        response = []
        ids = split_list(ids)
        for i in ids:
            i = str(i)[1:-1].replace(" ", "")
            print(i)
            res = requests.get(
                "https://api.vk.com/method/users.get",
                params={
                    "user_ids": i,
                    "fields": "nickname, screen_name, sex, bdate, city, country, timezone, photo, photo_medium",
                    "access_token": TOKEN
                })
            if res.status_code == 200:
                if "error" in res.json():
                    print(res.json()["error"]["error_code"])
                else:
                    response.extend(res.json()["response"])
            else:
                print("Ошибка сервера", res.status_code)
        return response
    else:
        raise ValueError("Функция принимает на вход список id")
r_u = get_users(friends)

1221,2050,2786,3327,4206,6486,9327,10942,12281,12477,15436,18528,21720,22529,23424,24608,26164,30231,35798,35890,35978,36423,39093,39129,41189,47072,48523,49333,50805,53461,57479,63142,64372,71462,75810,80420,91603,95168,97471,100490,107578,107685,108149,122949,124164,125990,126820,129284,136012,142701,154013,155584,162449,162772,163016,163598,164035,174082,180827,185680,189155,190324,197394,201799,212744,228537,232496,233970,237442,241907,276272,287568,290669,301527,302881,307489,312498,322020,325860,331881,335060,357673,370042,370771,414845,423646,425644,437714,443475,452269,452643,458918,475235,483980,489556,500323,507257,517809,525706,526190
532387,542984,545009,546265,547419,548163,584135,584475,585004,638027,642667,648224,661844,668369,668413,675985,709431,712698,751802,752181,753503,761134,761792,770150,770885,774508,781837,791424,808403,817576,831926,862546,894731,895283,897257,900022,904769,922058,922571,924061,924925,957107,973277,984502,1005246,1013022,1023140,1068580,107156

In [70]:
r_u

[{'bdate': '8.5.1977',
  'city': 2,
  'country': 1,
  'first_name': 'Igor',
  'last_name': 'Larionov',
  'nickname': '',
  'photo': 'https://pp.userapi.com/c638525/v638525683/63637/QWBZYCXt6BI.jpg',
  'photo_medium': 'https://pp.userapi.com/c638525/v638525683/63636/h0wdVhBIkrc.jpg',
  'screen_name': 'id1221',
  'sex': 2,
  'uid': 1221},
 {'bdate': '27.4',
  'city': 2,
  'country': 1,
  'first_name': 'Katya',
  'last_name': 'Lebedeva',
  'nickname': '',
  'photo': 'https://pp.userapi.com/c625717/v625717050/4e6d8/zBFozNxx9Ec.jpg',
  'photo_medium': 'https://pp.userapi.com/c625717/v625717050/4e6d7/BWpyJXdJ7Rs.jpg',
  'screen_name': 'me',
  'sex': 1,
  'uid': 2050},
 {'bdate': '22.7',
  'city': 1,
  'country': 1,
  'first_name': 'Leonid',
  'last_name': 'Savintsev',
  'nickname': '',
  'photo': 'https://pp.userapi.com/c841624/v841624538/2126b/SRWtbVaiiZE.jpg',
  'photo_medium': 'https://pp.userapi.com/c841624/v841624538/2126a/LdjhuS_gEd4.jpg',
  'screen_name': 'lsavintsev',
  'sex': 2,
  '